In [5]:
from arcgis.gis import GIS
import arcgis.network as network
from arcgis.features import FeatureLayer, Feature, FeatureSet, use_proximity, FeatureCollection
import pandas as pd
import time
import datetime as dt

In [8]:
my_gis = GIS('https://www.arcgis.com', 'srivathskm', 'Hedebyhedge7$', verify_cert=False, set_active=True)

In [10]:
import requests
import csv
import os

out_file_name = 'hospitals_SB_counties.csv'
url = "https://data.chhs.ca.gov/datastore/dump/641c5557-7d65-4379-8fea-6b7dedbda40b?q=&sort=_id+asc&fields=OSHPD_ID%2CFACILITY_NAME%2CLICENSE_NUM%2CFACILITY_LEVEL_DESC%2CDBA_ADDRESS1%2CDBA_CITY%2CDBA_ZIP_CODE%2CCOUNTY_CODE%2CCOUNTY_NAME%2CER_SERVICE_LEVEL_DESC%2CTOTAL_NUMBER_BEDS%2CFACILITY_STATUS_DESC%2CFACILITY_STATUS_DATE%2CLICENSE_TYPE_DESC%2CLICENSE_CATEGORY_DESC%2CLATITUDE%2CLONGITUDE&filters=%7B%22COUNTY_CODE%22%3A+%5B%2236%22%5D%7D&format=csv"
download = requests.get(url)

with open(out_file_name, 'w') as out_file:
    out_file.writelines(download.text)
    print(out_file_name)
csv_item = my_gis.content.add({}, out_file_name)
hospital_item = csv_item.publish()
display(hospital_item)

hospitals_SB_counties.csv


<Item title:"hospitals_SB_counties" type:Feature Layer Collection owner:srivathskm>

In [11]:
hospital_fl = hospital_item.layers[0]
hospital_fl

<FeatureLayer url:"https://services.arcgis.com/8r5Y8U62UfdJkAyA/arcgis/rest/services/hospitals_SB_counties/FeatureServer/0">

In [12]:
facilities = hospital_fl.query(where="Dba_city='REDLANDS'", as_df=False)
display(facilities)

<FeatureSet> 27 features

In [13]:
incidents_json = {
                    "features": [{"attributes": {"CurbApproach": 0,
                                                 "ID": "C100045",
                                                 "Name": "Incident at Esri"},
                                  "geometry": {"x": -117.19569523299998, "y": 34.05608640000003}},
                                 {"attributes": {"CurbApproach": 0,
                                                 "ID": "F100086",
                                                 "Name": "Incident at APT"},
                                  "geometry": {"x": -117.20520037855628, "y": 34.04472649163186}},
                                 {"attributes": {"CurbApproach": 0,
                                                 "ID": "C100097",
                                                 "Name": "Incident at Walmart"},
                                  "geometry": {"x": -117.222253, "y": 34.065378}}],
                    "spatialReference": {"wkid": 4326, "latestWkid": 4326},
                    "geometryType": "esriGeometryPoint",
                    "fields" : [
                        {"name" : "ID", "type" : "esriFieldTypeString", "alias" : "ID", "length" : "50"},
                        {"name" : "Name", "type" : "esriFieldTypeString", "alias" : "Name", "length" : "50"},
                        {"name" : "CurbApproach", "type" : "esriFieldTypeInteger", "alias" : "CurbApproach"}
                    ]
                }
incidents = FeatureSet.from_dict(incidents_json)

In [14]:
incidents

<FeatureSet> 3 features

In [15]:
map1 = my_gis.map('Redlands, CA', zoomlevel=12)
map1

MapView(layout=Layout(height='400px', width='100%'))

In [19]:
hospital_symbol = {"type":"esriPMS",
                   "url":"http://static.arcgis.com/images/Symbols/SafetyHealth/Hospital.png",
                   "contentType": "image/png", "width":20, "height":20}

map1.draw(facilities, symbol=hospital_symbol)

In [18]:
traffic_accident_symbol = {"type":"esriPMS",
                           "url":"http://static.arcgis.com/images/Symbols/Transportation/TrafficAccident.png",
                           "contentType": "image/png", "width":20, "height":20}

map1.draw(incidents, symbol=traffic_accident_symbol)

In [55]:
%%time
current_time = dt.datetime.now()  
result1 = network.analysis.find_closest_facilities(incidents=incidents, facilities=facilities, 
                                                   cutoff=10, time_of_day=current_time, 
                                                   number_of_facilities_to_find=4,
                                                   gis=my_gis, populate_directions=True, save_output_network_analysis_layer=True, save_route_data=True)

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").


CPU times: user 155 ms, sys: 15.7 ms, total: 170 ms
Wall time: 9.83 s


In [56]:
result1

ToolOutput(output_routes=<FeatureSet> 12 features, output_directions=<FeatureSet> 85 features, solve_succeeded=True, output_closest_facilities=<FeatureSet> 12 features, output_network_analysis_layer={"url": "https://logistics.arcgis.com/arcgis/rest/directories/arcgisjobs/world/closestfacility_gpserver/j09ae54d56c834a869ff8ebc0fa6bdd63/scratch/_ags_gpna4f8cbc79543e4435914cedaa3dcc3c8f.lyr"}, output_route_data={"url": "https://logistics.arcgis.com/arcgis/rest/directories/arcgisjobs/world/closestfacility_gpserver/j09ae54d56c834a869ff8ebc0fa6bdd63/scratch/_ags_rd0e7f4969550242319bdfa641431163f7.zip"}, output_incidents=<FeatureSet> 3 features, output_facilities=<FeatureSet> 27 features, output_result_file=None)

In [61]:
df4 = result1.output_routes.sdf[["Name", "IncidentID", "Total_Kilometers", "Total_Minutes"]]

In [26]:
map2 = my_gis.map('Redlands, CA', zoomlevel=12)
map2

MapView(layout=Layout(height='400px', width='100%'))

In [37]:
map2.draw(facilities, symbol=hospital_symbol, attributes={"title":"Hospital Facility"})
map2.draw(incidents, symbol=traffic_accident_symbol, attributes={"title":"Incident Location"})

In [60]:
map2.add_layer(result1.output_routes)

In [42]:
directions = result1.output_directions.sdf

In [59]:
routed_directions = directions.groupby("RouteName").Text.apply(list)

In [79]:
routed_directions = routed_directions.iloc[[2, 10, 6]]

RouteName
Incident at APT - Location 2         [Start at Incident at APT, Go east on Brooksid...
Incident at Walmart - Location 22    [Start at Incident at Walmart, Go north, Restr...
Incident at Esri - Location 26       [Start at Incident at Esri, Go west toward New...
Name: Text, dtype: object

In [88]:
facilies = (result1.output_facilities.sdf['FACILITY_NAME'])

In [90]:
facilies[0]

'REDLANDS COMMUNITY HOSPITAL'

In [62]:
df4_shortest = df4.sort_values("Total_Minutes").groupby("IncidentID", as_index=False).first()
df4_shortest[["Name", "IncidentID", "Total_Kilometers", "Total_Minutes"]]

,Name,IncidentID,Total_Kilometers,Total_Minutes
0,Incident at Esri - Location 26,C100045,0.861167,1.374616
1,Incident at Walmart - Location 22,C100097,0.628640,1.780832
2,Incident at APT - Location 2,F100086,0.431715,0.575727


In [97]:
int(df4_shortest.Name[0].split()[-1])

'26'

In [86]:
with_route = df4_shortest.merge(routed_directions, left_on='Name', right_on="RouteName")

In [105]:
dests = []
for index, row in with_route.iterrows():
    number = int(df4_shortest.Name[index].split()[-1])
    dests.append(facilies[number])

In [106]:
with_route["Destination"] = dests

In [111]:
with_route.rename(columns={"Text":"Directions"})

,IncidentID,Name,Total_Kilometers,Total_Minutes,Directions,Destination
0,C100045,Incident at Esri - Location 26,0.861167,1.374616,"[Start at Incident at Esri, Go west toward New...","BRIGHT SKY HOME HEALTH CARE, INC."
1,C100097,Incident at Walmart - Location 22,0.628640,1.780832,"[Start at Incident at Walmart, Go north, Restr...",PRIOR LEVEL HOME HEALTH
2,F100086,Incident at APT - Location 2,0.431715,0.575727,"[Start at Incident at APT, Go east on Brooksid...",LOMA LINDA UNIVERSITY SURGICAL HOSPITAL


In [ ]:
with_route['Text'].rename()

In [68]:
red_highlight_symbol = {'type': 'esriSLS', 'style': 'esriSLSSolid','color': [237, 0, 0, 100], 'width': 6}
map2.draw(df4_shortest.spatial.to_featureset(), symbol=red_highlight_symbol)
map2.zoom=14

TypeError: 'NoneType' object does not support item assignment